In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [4]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [7]:
len(model.encode('hello!, my name is miguel'))

384

In [8]:
#En este caso queremos 3 embeddings, embedding de la pregunta, embedding de la respuesta, y embedding de la combinación pregunta+respuesta

In [9]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
from tqdm.auto import tqdm

In [11]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)
    


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [01:53<00:00,  8.38it/s]


In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.00it/s]


In [13]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [17]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']
    

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [18]:
import pandas as pd

In [19]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [20]:
df_ground_truth.head()


,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


In [21]:
ground_truth = df_ground_truth.to_dict(orient='records')


In [22]:
def hit_rate(relevance_total):
    count = 0
    for line in relevance_total:
        if True in line:
            count +=1
    return count/len(relevance_total)

In [32]:
#MMR no solo se fija en si está el valor o no, sino también en la posición.

In [23]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [24]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [25]:
evaluate(ground_truth, question_vector_knn)

#Elastic search question evaluation


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:39<00:00, 46.43it/s]


{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

In [26]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']
    

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [27]:
evaluate(ground_truth, text_vector_knn)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:36<00:00, 47.71it/s]


{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

In [28]:
def qt_vector_knn(q):
    question = q['question']
    course = q['course']
    

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [29]:
evaluate(ground_truth, qt_vector_knn)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:35<00:00, 48.25it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}